In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

num_bins=50

# I. Load and plot

In [ ]:
Scenario    = 'Case1'
path_prefix = './data/' + Scenario + '_'

# Loading data
with open(path_prefix + 'first_kind.npy', 'rb') as f:
    zeroes_first_kind = np.load(f)
with open(path_prefix + 'second_kind.npy', 'rb') as f:
    zeroes_second_kind = np.load(f)
with open(path_prefix + 'critical_points.npy', 'rb') as f:
    critical_points = np.load(f)
with open(path_prefix + 'branch_points.npy', 'rb') as f:
    branch_points = np.load(f)

N = len( zeroes_first_kind )

# Histogram of singular values
fig, ax = plt.subplots()
n, bins, patches = ax.hist(zeroes_first_kind, num_bins, density=True)
ax.set_xlabel('Eigenvalues')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of singular values for N={}'.format(N))
fig.tight_layout()
plt.show()

In [ ]:
# Plotting critical points
plt.figure( figsize=(20,10) )
plt.scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
plt.scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
plt.scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
plt.title( 'Eigenvalues and critical points')
plt.legend()
plt.show()

# Plotting branch points
## Transport colors to uniform
branch_points_color = np.abs( np.imag( critical_points ) )
branch_points_color /= np.max( branch_points_color )
uniform_sample = np.linspace( 0, 1, len(branch_points_color))
sorted_indices = np.argsort( branch_points_color )
branch_points_color = uniform_sample[ sorted_indices ]
## Plots as usual
plt.figure( figsize=(10,20) )
plt.scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
plt.scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
plt.title( 'Branch points $m = M_\mu(z)$')
plt.legend()
plt.colorbar()
plt.show()

# II. Performance test when evaluating Stieljes transforms

Here we compare numpy vs sympy. Although it may be true that symbolic computations
using sympy might be slower, the question is to evaluate the overhead.

In [ ]:
def G_empirical(z):
    array = z[...,None]-zeroes_first_kind[...,:]
    return np.sum( 1/array, axis=-1)/len(zeroes_first_kind)

def G_prime_empirical(z):
    array = z[...,None]-zeroes_first_kind[...,:]
    return np.sum( -1/(array*array), axis=-1)/len(zeroes_first_kind)

def G_second_empirical(z):
    array = z[...,None]-zeroes_first_kind[...,:]
    return np.sum( 2/(array*array*array), axis=-1)/len(zeroes_first_kind)

def M_empirical(z):
    return z*G_empirical(z)-1

def M_prime_empirical(z):
    return z*G_prime_empirical(z) + G_empirical(z)

def M_second_empirical(z):
    return z*G_second_empirical(z) + 2*G_prime_empirical(z)

## III. Inversion without homotopy

In [ ]:
def newton_raphson( f, f_prime, initial, max_iter=12, tol=1e-12):
    z = initial
    k = 0
    while k<max_iter:
        value = f(z)
        derivative_value = f_prime(z)
        z = z - value/derivative_value
        if abs(value)<tol:
            break
        k = k + 1
    return z

In [ ]:
# Find the z_target such that M(z_target) = m_target
empirical_mean = zeroes_first_kind.mean()
m_array = np.linspace( 0.1, 20.0, 1000 )*(-1j) # Breaks at 0.7
z_array = []
debug = False
for m in m_array:
    m_target  = np.array( [m] )
    if len(z_array)>0:
        z_initial = z_array[-1]
    else:
        z_initial = empirical_mean/m_target
    z_result  = newton_raphson( lambda z : M_empirical(z)-m_target, M_prime_empirical, z_initial)
    z_array.append( z_result )
    if debug:
        print( "Result: ", z_result)
        print( "Error: ", np.abs( m_target - M_empirical(z_result)) )
z_array = np.array( z_array )

m_final = m_array[-1]
z_final = z_array[-1]

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].set_title( 'm space')

plt.legend()
plt.show()

In [ ]:
# Find the z_target such that M(z_target) = m_target
empirical_mean = zeroes_first_kind.mean()
radius = 1.4
phase  = -np.pi/2 
interval = np.linspace( phase, phase + 2*np.pi, 100 )
ellipse  = np.cos( interval ) + 1.0*np.sin( interval )*1j
m_array = radius*ellipse
z_array = []
debug = False
for m in m_array:
    m_target  = np.array( [m] )
    z_initial = empirical_mean/m_target
    if len(z_array)>0:
        z_initial = z_array[-1]
    else:
        z_initial = z_final
    z_result  = newton_raphson( lambda z : M_empirical(z)-m_target, M_prime_empirical, z_initial)
    z_array.append( z_result )
    if debug:
        print( "Result: ", z_result)
        print( "Error: ", np.abs( m_target - M_empirical(z_result)) )
z_array = np.array( z_array )

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(critical_points), -np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
axs[1].scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
axs[1].set_title( 'm space')
axs[1].set_ylim( -5, 5)

plt.legend()
plt.show()

In [ ]:
from freeDeconvolution import boxes

def box_to_path( box, mesh_size):
       interval =  np.linspace( 0,1, mesh_size)
       path = []
       for segment in boxes.box_segments_enum:
              vector = box[ segment[1] ] - box[ segment[0] ]
              origin = box[ segment[0] ]
              s = origin + interval*vector
              #
              path = path + list(s)
       return path

radius = np.max( np.imag(critical_points) )*1.2
box = {
       'top_left'    : np.min( zeroes_first_kind ) - 0.5 + radius*1.0j,
       'bottom_right': np.max( zeroes_first_kind ) + 0.5 - radius*1.0j,
}
bounding_box = boxes.extend_box( box )
path = box_to_path( box, int(1e5) )
plt.plot( np.real(path), np.imag(path), c='r')
plt.show()

z_array = np.array( path )
m_array = M_empirical( z_array )

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(critical_points), -np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
axs[1].scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
axs[1].set_title( 'm space')
axs[1].set_ylim( -5, 5)

plt.legend()
plt.show()

In [ ]:
s_array = (1+m_array)/(m_array*z_array)

if Scenario=='Case1':
    c_interval = np.linspace(0.1, 0.5, 100)
    error_array = []
    for c_ in c_interval:
        error = np.abs( s_array*(c_*m_array+1)-1).max()
        error_array.append( error )
    error_array = np.array( error_array )
    plt.plot( c_interval, error_array )
    plt.show()

### Formulas

$$ m = zg - 1$$
$$ s = \frac{1+m}{m z}
   \Leftrightarrow
   sz - 1 = \frac{1}{m}
$$

In [ ]:
if Scenario=='Case1':
    c=0.3
elif Scenario=='Case2':
    c=0.3
elif Scenario=='Case3':
    c=0.2

s_signal_array = s_array
s_noise_array  = 1/(c*m_array + 1)
s_deconv_array = s_signal_array/s_noise_array
m_deconv_array = 1/( s_deconv_array*z_array - 1)
m_deconv_theoretical_array = 1/(z_array-1)

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(critical_points), -np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_deconv_array), np.imag(m_deconv_array), label=f'Empirical path in m space')
axs[1].plot( np.real(m_deconv_theoretical_array), np.imag(m_deconv_theoretical_array), label=f'Ground truth in m space')
axs[1].scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
axs[1].scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
axs[1].set_title( 'm space')
axs[1].set_ylim( -7, 7)

plt.legend()
plt.show()

# IV. Deconvolution 

In [ ]:
g_deconv_array = (m_deconv_array+1)/z_array
g_deconv_theoretical = (m_deconv_theoretical_array+1)/z_array

space_array   = np.linspace(-1, 7, 100)
resolution = 10.0
dz_array = z_array-np.roll(z_array, shift=1)
empirical_cdf = []
for x in space_array:
    sigmoid = 1/(1+np.exp(resolution*(z_array-x)))
    value = g_deconv_array*sigmoid*dz_array
    value = value.sum()/(2*np.pi*1.0j)
    empirical_cdf.append( value )
# end for
empirical_cdf = np.array( empirical_cdf )

plt.figure( figsize=(10,5) )
plt.plot( space_array, space_array>1.0, label='Asymptotic ground truth = Theoretical cdf of deconvolved signal' )
plt.plot( space_array, np.real(empirical_cdf), label=f'''Empirical cdf of deconvolved signal and blur at resolution {1/resolution}''' )
plt.plot( space_array+0.03, 1/(1+np.exp(resolution*(1-space_array))), label='Theoretical cdf of deconvolved signal at same resolution' )
plt.legend()
plt.show()

# V. OPRL reconstruction

In [ ]:
# Compute moments
# Input: z_array, g_deconv_array

moments_count = 5
dz_array  = z_array-np.roll(z_array, shift=1)
print( z_array )
mom_array = np.zeros( 2*moments_count )
for mom_index in range(2*moments_count):
    value = g_deconv_array*(z_array**mom_index)*dz_array
    value = value.sum()/(2*np.pi*1.0j)
    print(value)
    mom_array[ mom_index ] = np.real(value)
# end for
print( mom_array )

In [ ]:
# Form moment matrix
from scipy.linalg import cholesky as scipy_cholesky

mom_matrix = np.zeros( shape=(moments_count, moments_count) )
for index in range( moments_count ):
    mom_matrix[index,:] = mom_array[index:(index+moments_count)]

# Cholesky
try:
    cholesky = scipy_cholesky( mom_matrix )
except np.linalg.LinAlgError as err:
    print( err  )
    print("")

# Retry
mom_count = 1
print(f'''Retrying with {mom_count}...''')
try:
    cholesky = scipy_cholesky( mom_matrix[0:mom_count, 0:mom_count], lower=False )
    print("Passed")
except np.linalg.LinAlgError as err:
    print( err  )

print( cholesky )


In [ ]:
mom_count = cholesky.shape[0]
diag_indices  = np.array( np.diag_indices( mom_count ) )
extra_indices = diag_indices.copy()[:, :-1]
extra_indices[1,:] += 1
print( diag_indices )
print( extra_indices )
diag_cholesky = cholesky[diag_indices[0,:] , diag_indices[1,:]]
extra_diag    = cholesky[extra_indices[0,:], extra_indices[1,:]]
print( diag_cholesky )
print( extra_diag )
print("")

# Compute Jacobi
jacobi_b = diag_cholesky[1:]/diag_cholesky[:-1]
jacobi_a = np.zeros_like( diag_cholesky )
jacobi_a[0] = diag_cholesky[0]
x_over_y = extra_diag/diag_cholesky[1:]
jacobi_a[1:-1] = x_over_y[1:]-x_over_y[:-1]
# Print Jacobi coefficients
print( "Jacobi coefficients:")
print( "b: ", jacobi_b )
print( "a: ", jacobi_a )
print( "")

# Form Jacobi matrix
jacobi = np.zeros( shape=(mom_count, mom_count) )
jacobi[extra_indices[0,:], extra_indices[1,:]] = jacobi_b
jacobi = jacobi + jacobi.T + np.diag( jacobi_a )
print( "Jacobi matrix ")
print( jacobi )
print( "" )

# Eigenvalues
print("Eigenvalues")
eigen, vectors = np.linalg.eig(jacobi)
eigen = np.sort(eigen)
print(eigen)
# Compute weights
vandermonde = np.vander( eigen ).T
weights = np.linalg.solve( vandermonde, mom_array[0:mom_count] )
print( "Raw Weights: ")
print( weights)

In [ ]:
# Discard negative weights and eigenvalues outside of support
min_zero = np.min( zeroes_first_kind )
max_zero = np.max( zeroes_first_kind )
admissible_eigen   = np.where( (eigen >= min_zero) *  (eigen <= max_zero) * (weights > 0) )[0]
admissible_weights = weights[ admissible_eigen ]
admissible_weights = admissible_weights/np.sum( admissible_weights )
admissible_eigen   = eigen[ admissible_eigen ]
print("Admissible weights: ", admissible_weights)
print("Admissible eigen  : ", admissible_eigen)

# Plot
space_array   = np.linspace(-1, 7, 100)
empirical_cdf = np.zeros_like( space_array )
for index in range(len(admissible_eigen)):
    eigenvalue = admissible_eigen[ index ]
    empirical_cdf += admissible_weights[index] * ( space_array >= eigenvalue )
# end for
empirical_cdf = np.array( empirical_cdf )

plt.figure( figsize=(10,5) )
plt.plot( space_array, space_array>1.0, label='Asymptotic ground truth = Theoretical cdf of deconvolved signal' )
plt.plot( space_array, np.real(empirical_cdf), label=f'''Empirical cdf of deconvolved signal with OPRL reconstruction''' )
plt.legend()
plt.show()
